In [1]:
!pip install transformers datasets trl huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [7]:
import torch
import os
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import pandas as pd
from datasets import Dataset
import os

In [3]:
data_path = 'User_Recommendations.csv'
df = pd.read_csv(data_path)
dataset = Dataset.from_pandas(df)

In [4]:
def process_data_for_dpo(example):
    messages = [{"role": "user", "content": f"Recommend a food list for a person whose gender is {example['RIAGENDR']}, age: {example['RIDAGEYR']}, hypertension: {example['BPQ020']}, diabetes: {example['DIQ010']}"},
                {"role": "assistant", "content": ", ".join(eval(example['Recommended Foods']))}]
    return {"messages": messages}

In [5]:
processed_dataset = dataset.map(process_data_for_dpo)

Map:   0%|          | 0/6180 [00:00<?, ? examples/s]

In [8]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Load the model and tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Set up the chat format
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

model.config.use_cache = False

# Set our name for the finetune to be saved &/ uploaded to
finetune_name = "SmolLM2-FT-MyDataset"
finetune_tags = ["smol-course", "module_1"]

In [9]:
os.environ["WANDB_DISABLED"] = "true"

# Configure the SFTTrainer
sft_config = SFTConfig(
    output_dir="./sft_output",
    max_steps=1000,  # Adjust based on dataset size and desired training duration
    per_device_train_batch_size=4,  # Set according to your GPU memory capacity
    learning_rate=5e-5,  # Common starting point for fine-tuning
    logging_steps=10,  # Frequency of logging training metrics
    save_steps=100,  # Frequency of saving model checkpoints
    # evaluation_strategy="steps",  # Evaluate the model at regular intervals
    # eval_steps=50,  # Frequency of evaluation
)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=processed_dataset,
    tokenizer=tokenizer,
    # eval_dataset=ds["test"],
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/6180 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [10]:
# Train the model
trainer.train()

# Save the model
trainer.save_model(f"./{finetune_name}")

Step,Training Loss
10,2.198400
20,0.927100
30,0.864300
40,0.823000
50,0.821300
60,0.805000
70,0.818400
80,0.759700
90,0.777600
100,0.717600


In [11]:
prompt = "Recommend a food list for a person whose gender is 1, age: 62, hypertension: 2, diabetes: 1"
test_prompt = [{"content": prompt, "role": "user"}]

formatted_prompt = tokenizer.apply_chat_template(test_prompt, tokenize=False, add_generation_prompt=True)
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

In [12]:
outputs = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_length=100)
print("Model Çıktısı:", tokenizer.decode(outputs[0], skip_special_tokens=True))

Model Çıktısı: user
Recommend a food list for a person whose gender is 1, age: 62, hypertension: 2, diabetes: 1
assistant
Coconut, Pork chop, Pork leg, Pork shoulder
Pork chop, Pork leg, Pork shoulder, Pork leg, Pork leg, Pork leg, Pork leg, Pork leg, Pork leg, Pork leg, Pork leg, Pork leg
